## Generate TRAPPIST-1 data

This notebook is designed to generate the main dataset used in the analysis, including application of filters. Here, the magma emplacement rate is randomly sampled for each TRAPPIST-1 planet.

*Important*: The VolcGases library is required to run this notebook and can be found at `https://github.com/Nicholaswogan/VolcGases`

In [ ]:
import numpy as np
import pandas as pd
from VolcGases.functions import solve_gases

molar_keys = ['H2O','H2','CO2','CO','CH4']
molar_vals = np.array([18.015,2.016,44.01,28.01,16.04])*1e-3
molar_masses = dict(zip(molar_keys, molar_vals))

## Functions that do the core calculations

In [2]:
# This does the calculation of the equilibrium degassing model from Wogan et al. 2020
def outgas(input_dict):
        
    m_H2O_mantle = input_dict["m_H2O_mantle"]
    F = input_dict["F"]
    Pprim = input_dict["Pprim"]
    T = input_dict["T"]
    x_CO2_melt = input_dict["x_CO2_melt"]
    FMQ = input_dict["FMQ"]
    Psurf = input_dict["Psurf"]
    
    D = 0.01
    A = 25738
    B = 9
    C = 0.092
    x = 0.01550152865954013
    Q_earth = 9e13
    
    # Step 1: Compute melt water content
    m_frac = (m_H2O_mantle/F)*(1-np.power((1-F),1/D))
    m_sat  = ((12*np.power(Pprim,0.6)) + (1*Pprim))/100
    x_H2O_melt = np.min([m_frac,m_sat])

    # Step 2: Compute gas production
    log_FMQ = (-A/T+B+C*(Psurf-1)/T)
    f_O2 = 10**(log_FMQ+FMQ)

    P_H2O,P_H2,P_CO2,P_CO,P_CH4,alphaG,x_CO2,x_H2O = solve_gases(T,Psurf,f_O2,x_CO2_melt,x_H2O_melt)
    H2O_gas = 1000*alphaG*x*(1/(1-alphaG))*P_H2O/Psurf # mol per kg
    H2_gas = 1000*alphaG*x*(1/(1-alphaG))*P_H2/Psurf # mol per kg
    CO2_gas = 1000*alphaG*x*(1/(1-alphaG))*P_CO2/Psurf # mol per kg
    CO_gas = 1000*alphaG*x*(1/(1-alphaG))*P_CO/Psurf # mol per kg
    CH4_gas = 1000*alphaG*x*(1/(1-alphaG))*P_CH4/Psurf # mol per kg
    
    keys = ['H2O_gas', 'H2_gas', 'CO2_gas', 'CO_gas', 'CH4_gas']
    vals = [H2O_gas, H2_gas, CO2_gas, CO_gas, CH4_gas]
    output_dict = dict(zip(keys, vals))
    
    return output_dict

# This is for calculating the outgassing fluxes of the species given the result of the equil degas code and Q
def calcFluxes_KgYr(Q,outgas_dict):
    
    # mag_rho = 2900*1e9 # kg km-3, density of magma, consistent with Earth oceanic crust
    # molar_keys = ['H2O','H2','CO2','CO','CH4']
    # molar_vals = np.array([18.015,2.016,44.01,28.01,16.04])*1e-3
    # molar_masses = dict(zip(molar_keys, molar_vals))
    
    H2O_flux = Q*outgas_dict['H2O_gas']*18.015*1e-3*2900*1e9
    H2_flux  = Q*outgas_dict['H2_gas'] *2.016*1e-3*2900*1e9
    CO2_flux = Q*outgas_dict['CO2_gas']*44.01*1e-3*2900*1e9
    CO_flux  = Q*outgas_dict['CO_gas'] *28.01*1e-3*2900*1e9
    CH4_flux = Q*outgas_dict['CH4_gas']*16.04*1e-3*2900*1e9
    
    keys = ['H2O_flux', 'H2_flux', 'CO2_flux', 'CO_flux', 'CH4_flux']
    vals = [H2O_flux, H2_flux, CO2_flux, CO_flux, CH4_flux]
    output_dict = dict(zip(keys, vals))
    
    return output_dict

# This is for randomly sampling parameters
def drawParams():
    
    m_H2O_mantle = np.random.uniform(0.01/100,3/100)
    F            = np.random.uniform(10/100,20/100)
    Pprim        = np.random.uniform(1,3)
    T            = np.random.uniform(873,1973)
    x_CO2_melt   = np.power(10,np.random.uniform(-5,-2))
    FMQ          = np.random.uniform(-4,5)
    Psurf        = np.power(10,np.random.uniform(-3,2))
    
    keys = ['m_H2O_mantle', 'F', 'Pprim', 'T', 'x_CO2_melt', 'FMQ', 'Psurf']
    vals = [m_H2O_mantle, F, Pprim, T, x_CO2_melt, FMQ, Psurf]
    output_dict = dict(zip(keys, vals))
    
    return output_dict

## Execute the parameter sweep

In [3]:
NSAMPLES = int(1e5)

input_names = ["m_H2O_mantle","F","Pprim","T","x_CO2_melt","FMQ","Psurf"]
result_names = ["H2O_gas", "H2_gas", "CO2_gas", "CO_gas", "CH4_gas"]
flux_names = ["Q","H2O_flux","H2_flux","CO2_flux","CO_flux","CH4_flux"]
all_column_names = input_names + result_names + flux_names


minQ = 0.01
maxQs = [8500, 5840, 1230, 2159, 2130, 2344, 1230]

t1_dfs = np.empty(7,dtype=object)


for i in range(len(t1_dfs)):
    outgas_df = pd.DataFrame(columns=all_column_names, index=range(0, NSAMPLES))
    maxQ = maxQs[i]
    
    for j in range(NSAMPLES):
        loop_inp = drawParams() # sample the input parameters
        loop_res = outgas(loop_inp) # calculate the outgassing fluxes

        Q = np.power(10,np.random.uniform(np.log10(minQ),np.log10(maxQ))) # log sample the magma emplacement rate, up to the maximum Q across all of T1
        loop_flux = calcFluxes_KgYr(Q,loop_res)

        outgas_df.iloc[j] = list(loop_inp.values()) + list(loop_res.values()) + [Q] + list(loop_flux.values())
    
    t1_dfs[i] = outgas_df

## Calculate filter 1

In [4]:
# CALCULATE ENERGY LIMITED HYDROGEN ESCAPE RATE

# T1 planet values
r_earth = 6371*1e3 # m
rp = np.array([1.116, 1.097, 0.788, 0.920, 1.045, 1.129, 0.755])*r_earth # m

M_earth = 5.9722e24 # kg
Mp = np.array([1.374, 1.308, 0.388, 0.692, 1.039, 1.321, 0.326])*M_earth # kg

Fxuv = np.array([2935.13,1564.87,788.455,456.452,263.598,178.116,102.035])*1e-7 # joules s-1 cm-2

# Constants in the equation
epsilon = 0.3 # unitless
G = 6.674e-11 # N m2 kg-2
Ktide = 1 # unitless
mH = 1*1.67377e-27 # kg
sa = 4*np.pi*np.power(rp,2)

# Final equation
phi_cgs = (epsilon*Fxuv*rp)/(4*G*Mp*mH) # H atoms s-1 cm-2
phi_final = phi_cgs*(1.67377e-27)*(3.1536e7)*(sa*1e4) # kg yr-1, H2_FLUX is not allowed to be higher than this

## Calculate filter 2

In [5]:
# CALCULATE THE UPPER RATE LIMIT FOR EACH PLANET GIVEN A CAP ON WMF

max_wmf = 3/100
# t1_age = 9.8e9 # yr
t1_age = 5.4e9 # yr

cmf = 0.18
M_mantle = (1-cmf)*Mp

temp_m_h2o_mantle = 0.01
# max_rate = (M_mantle[0]*(max_wmf - temp_m_h2o_mantle))/((1-max_wmf)*t1_age) # H2O_Flux is not allowed to be higher than this
# ^ This is just a place holder to see how it works, it can't be calculated beforehand

## Apply filters

In [6]:
planet_labels = ['b','c','d','e','f','g','h']

t1_dfs_filtered = np.empty(7,dtype=object)

# Loop through the 7 planets
for i in range(7):
    temp1 = t1_dfs[i] # Get the full sample for each planet
    filter_1_idxs = []
    for j in range(len(temp1)):
        h2_dominant = False
        above_limit = temp1.iloc[j]["H2_flux"] > phi_final[i]
        if above_limit:
            max_rate = np.max((temp1.iloc[j]["H2O_flux"]/molar_masses['H2O'],temp1.iloc[j]["CO2_flux"]/molar_masses['CO2'],temp1.iloc[j]["CO_flux"]/molar_masses['CO'],temp1.iloc[j]["CH4_flux"]/molar_masses['CH4']))
            h2_dominant = temp1.iloc[j]["H2_flux"]/molar_masses['H2'] > max_rate

        if not h2_dominant:
            filter_1_idxs.append(j)

    temp2 = temp1.iloc[filter_1_idxs]
    temp3 = temp2[temp2['H2O_flux'] <= (M_mantle[i]*(max_wmf - temp2['m_H2O_mantle']))/((1-max_wmf)*t1_age)] # Apply filter 2: interior evolution
    t1_dfs_filtered[i] = temp3
    
    pct_left = 100*temp3.shape[0]/temp1.shape[0]
    print(f"T1-{planet_labels[i]}: {pct_left:.2f}% remaining.")


T1-b: 61.97% remaining.
T1-c: 63.09% remaining.
T1-d: 61.09% remaining.
T1-e: 63.05% remaining.
T1-f: 66.36% remaining.
T1-g: 67.55% remaining.
T1-h: 59.63% remaining.


## Save data

In [7]:
np.save("sample_full",t1_dfs)
np.save("sample_filtered",t1_dfs_filtered)